# Infilling

In this notebook we demonstrate Silicone's infilling methods.

## Imports

In [1]:
# NBVAL_IGNORE_OUTPUT
import os.path
import traceback

import pandas as pd
import pyam
import matplotlib.pyplot as plt
import numpy as np

import silicone.database_crunchers

<IPython.core.display.Javascript object>

## Constants

In [2]:
SR15_SCENARIOS = "./sr15_scenarios.csv"

## Example data

Here we pull some example data by downloading a selection of the SR1.5 scenarios.

In [3]:
def get_sr15_scenarios(output_file):
    conn = pyam.iiasa.Connection("iamc15")
    valid_model_ids = [
        "MESSAGE*",
        "AIM*",
        "C-ROADS*",
        "GCAM*",
        "IEA*",
        "IMAGE*",
        #"MERGE*",
        #"POLES*",
        #"REMIND*",
        "WITCH*"
    ]
    variables_to_fetch = [
        "Emissions*",
    ]
    for model in valid_model_ids:
        print("Fetching data for {}".format(model))
        for variable in variables_to_fetch:
            print("Fetching {}".format(variable))
            var_df = conn.query(model=model, variable=variable, region='World')
            try:
                df.append(var_df, inplace=True)
            except NameError:
                df = pyam.IamDataFrame(var_df)

    print("Writing to {}".format(output_file))
    df.to_csv(output_file)

In [4]:
# NBVAL_IGNORE_OUTPUT
if not os.path.isfile(SR15_SCENARIOS):
    get_sr15_scenarios(SR15_SCENARIOS)

### Starting point

Our starting point is the test data, loaded a `pyam.IamDataFrame`.

In [5]:
# NBVAL_IGNORE_OUTPUT
sr15_data = pyam.IamDataFrame(SR15_SCENARIOS)
sr15_data.timeseries().head()

INFO:root:Reading `./sr15_scenarios.csv`


2000  \
model       scenario               region variable                                           unit      meta         
AIM/CGE 2.0 ADVANCE_2020_1.5C-2100 World  Emissions|BC                                       Mt BC/yr  0.0    NaN   
                                          Emissions|CH4                                      Mt CH4/yr 0.0    NaN   
                                          Emissions|CH4|AFOLU                                Mt CH4/yr 0.0    NaN   
                                          Emissions|CH4|Energy|Demand|Industry               Mt CH4/yr 0.0    NaN   
                                          Emissions|CH4|Energy|Demand|Residential and Com... Mt CH4/yr 0.0    NaN   

                                                                                                                 2005  \
model       scenario               region variable                                           unit      meta             
AIM/CGE 2.0 ADVANCE_2020_1.5C-2100 World  Emissions|BC                                       Mt BC/yr  0.0     8.3367   
                                          Emissions|CH4                                      Mt CH4/yr 0.0   344.9167   
                                          Emissions|CH4|AFOLU                                Mt CH4/yr 0.0   167.6717   
                                          Emissions|CH4|Energy|Demand|Industry               Mt CH4/yr 0.0     0.4732   
                                          Emissions|CH4|Energy|Demand|Residential and Com... Mt CH4/yr 0.0    11.2639   

                                                                                                                 2010  \
model       scenario               region variable                                           unit      meta             
AIM/CGE 2.0 ADVANCE_2020_1.5C-2100 World  Emissions|BC                                       Mt BC/yr  0.0     7.6408   
                                          Emissions|CH4                                      Mt CH4/yr 0.0   367.7457   
                                          Emissions|CH4|AFOLU                                Mt CH4/yr 0.0   179.6356   
                                          Emissions|CH4|Energy|Demand|Industry               Mt CH4/yr 0.0     0.5168   
                                          Emissions|CH4|Energy|Demand|Residential and Com... Mt CH4/yr 0.0    11.1249   

                                                                                                             2014  \
model       scenario               region variable                                           unit      meta         
AIM/CGE 2.0 ADVANCE_2020_1.5C-2100 World  Emissions|BC                                       Mt BC/yr  0.0    NaN   
                                          Emissions|CH4                                      Mt CH4/yr 0.0    NaN   
                                          Emissions|CH4|AFOLU                                Mt CH4/yr 0.0    NaN   
                                          Emissions|CH4|Energy|Demand|Industry               Mt CH4/yr 0.0    NaN   
                                          Emissions|CH4|Energy|Demand|Residential and Com... Mt CH4/yr 0.0    NaN   

                                                                                                                 2015  \
model       scenario               region variable                                           unit      meta             
AIM/CGE 2.0 ADVANCE_2020_1.5C-2100 World  Emissions|BC                                       Mt BC/yr  0.0     7.2724   
                                          Emissions|CH4                                      Mt CH4/yr 0.0   380.4086   
                                          Emissions|CH4|AFOLU                                Mt CH4/yr 0.0   190.3513   
                                          Emissions|CH4|Energy|Demand|Industry               Mt CH4/yr 0.0     0.5587   
                                          Emiss

## Crunchers

Silicone's 'crunchers' are used to determine the relationship between a 'follower variable' and 'lead variable(s)' from a given database. The 'follower variable' is the variable for which we want to generate data e.g. `Emissions|C3F8` while the 'lead variable(s)' is the variable we want to use in order to infer a timeseries of the 'follower variable'. The lead variable is typically CO$_2$ but might be a more complex combination of gases e.g. CO$_2$, CH$_4$ and N$_2$O. 

Each cruncher has a `derive_relationship` method. Its docstring describes what it does.

In [6]:
print(silicone.database_crunchers.base._DatabaseCruncher.derive_relationship.__doc__)
# # an alternative interface is shown if you use the line below instead
# silicone.database_crunchers.base._DatabaseCruncher.derive_relationship?


        Derive the relationship between two variables from the database

        Parameters
        ----------
        variable_follower : str
            The variable for which we want to calculate timeseries (e.g.
            ``"Emissions|C5F12"``).

        variable_leaders : list[str]
            The variables we want to use in order to infer timeseries of
            ``variable_follower`` (e.g. ``["Emissions|CO2", "Emissions|CH4"]``)

        **kwargs
            Keyword arguments used by this class to derive the relationship between
            ``variable_follower`` and ``variable_leaders``.

        Returns
        -------
        :obj:`func`
            Function which takes a :obj:`pyam.IamDataFrame` containing
            ``variable_leaders`` timeseries and returns timeseries for
            ``variable_follower`` based on the derived relationship between the two.
            Please see the source code for the exact definition (and docstring) of the
            returned functi

These crunchers are best explored by looking at the examples below.

### Closest RMS cruncher

This cruncher uses the 'closest RMS' methodology. It is described by its docstring.

In [7]:
print(silicone.database_crunchers.DatabaseCruncherRMSClosest.__doc__)
# silicone.database_crunchers.DatabaseCruncherRMSClosest?


    Database cruncher which uses the 'closest RMS' technkque.

    This cruncher derives the relationship between two variables by finding the
    scenario which has the closest lead gas timeseries in the database.  The follower
    gas timeseries is then simply copied from the closest scenario.

    Here, 'closest' is defined as the smallest time-averaged root mean squared (L2)
    difference.

    .. math::
        RMS = \left ( \frac{1}{n} \sum_{t=0}^n (E_l(t) - E_l^{d}(t))^2 \right )^{1/2}

    where :math:`n` is the total number of timesteps in the lead gas' timeseries,
    :math:`E_l(t)` is the lead gas emissions timeseries and :math:`E_l^d(t)` is a lead
    gas emissions timeseries in the database.
    


#### Infilling

Firstly, let's cut the database down to a size that is comprehensible.

In [8]:
sr15_data_closest_rms = sr15_data.filter(model=["WITCH-GLOBIOM 4.2"])

Now we can crunch the relationship between e.g. `Emissions|CO2` and `Emissions|VOC`.

In [9]:
cruncher = silicone.database_crunchers.DatabaseCruncherRMSClosest(sr15_data_closest_rms)

In [10]:
filler = cruncher.derive_relationship("Emissions|VOC", ["Emissions|CO2"])
print(filler.__doc__)
# filler?


            Filler function derived from :obj:`DatabaseCruncherRMSClosest`.

            Parameters
            ----------
            in_iamdf : :obj:`pyam.IamDataFrame`
                Input data to fill data in

            Returns
            -------
            :obj:`pyam.IamDataFrame`
                Filled in data (without original source data)

            Raises
            ------
            ValueError
                If there are any inconsistencies between the timeseries, units or
                expectations of the program and ``in_iamdf``, compared to the database
                used to generate this ``filler`` function.
            


Now we can do some infilling. As a sanity check, we firstly make sure that if we pass in a CO$_2$ timeseries which is already in the database, we get back its VOC emissions timeseries. 

In [11]:
example_model_scen = {
    "model": "WITCH-GLOBIOM 4.2",
    "scenario": "ADVANCE_INDC",
}
example_input = sr15_data_closest_rms.filter(**example_model_scen).data
example_input["model"] = "example"
example_input["scenario"] = "example"
example_input = pyam.IamDataFrame(example_input)

In [12]:
example_input_filled = filler(example_input)

In [13]:
example_input_filled.timeseries()

,,,,,,2005,2010,2015,2020,2025,2030,2035,2040,2045,2050,2055,2060,2065,2070,2075,2080,2085,2090,2095,2100
model,scenario,region,variable,unit,meta,,,,,,,,,,,,,,,,,,,,
example,example,World,Emissions|VOC,Mt VOC/yr,0.0,189.212,164.056724,153.575866,145.611005,130.387433,115.465871,102.038923,89.302542,76.60488,62.855685,58.440154,56.971497,55.075327,53.101911,51.074418,49.558506,47.602518,45.377459,43.08547,41.400122


In [14]:
sr15_data_closest_rms.filter(variable="Emissions|VOC", **example_model_scen).timeseries()

,,,,,,2005,2010,2015,2020,2025,2030,2035,2040,2045,2050,2055,2060,2065,2070,2075,2080,2085,2090,2095,2100
model,scenario,region,variable,unit,meta,,,,,,,,,,,,,,,,,,,,
WITCH-GLOBIOM 4.2,ADVANCE_INDC,World,Emissions|VOC,Mt VOC/yr,0.0,189.212,164.056724,153.575866,145.611005,130.387433,115.465871,102.038923,89.302542,76.60488,62.855685,58.440154,56.971497,55.075327,53.101911,51.074418,49.558506,47.602518,45.377459,43.08547,41.400122


As expected, the cruncher has picked out the scenario which matches and returned its VOC timeseries. This can also be seen in a plot.

In [ ]:
pkwargs = {"color": "scenario", "linewidth": 2}

fig = plt.figure(figsize=(16, 9))
ax = fig.add_subplot(121)
sr15_data_closest_rms.filter(variable="*CO2").line_plot(ax=ax, **pkwargs)
# sr15_data_closest_rms.filter(variable="*CO2", **example_model_scen).line_plot(ax=ax, color="black")
example_input.filter(variable="*CO2").line_plot(ax=ax, color="black", linestyle="--", dashes=(10, 15))
ax.set_title("Database CO$_2$ and lead emissions timeseries")

ax = fig.add_subplot(122)
sr15_data_closest_rms.filter(variable="*VOC").line_plot(ax=ax, **pkwargs)
example_input_filled.filter(variable="*VOC").line_plot(ax=ax, color="black", linestyle="--", dashes=(10, 15))
ax.set_title("Database VOC and filled follower emissions timeseries")

plt.tight_layout()

Now we can use our filler to infill other timeseries.

In [ ]:
filler_input = sr15_data.filter(model="MESSAGEix-GLOBIOM 1.0", scenario="CD-LINKS_NPi2020_1600")

This scenario is missing `Emissions|VOC`.

In [ ]:
filler_input.filter(variable="Emissions|VOC").data

In [ ]:
filler_input_filled = filler(filler_input)

In [ ]:
pkwargs = {"color": "scenario", "linewidth": 2}

fig = plt.figure(figsize=(16, 9))
ax = fig.add_subplot(121)
sr15_data_closest_rms.filter(variable="*CO2").line_plot(ax=ax, **pkwargs)
# sr15_data_closest_rms.filter(variable="*CO2", **example_model_scen).line_plot(ax=ax, color="black")
filler_input.filter(variable="*CO2").line_plot(ax=ax, color="black", linestyle="--", dashes=(10, 15))
ax.set_title("Database CO$_2$ and lead emissions timeseries")

ax = fig.add_subplot(122)
sr15_data_closest_rms.filter(variable="*VOC").line_plot(ax=ax, **pkwargs)
filler_input_filled.filter(variable="*VOC").line_plot(ax=ax, color="black", linestyle="--", dashes=(10, 15))
ax.set_title("Database VOC and filled follower emissions timeseries")

plt.tight_layout()

As we can see, on an RMS basis our input timeseries is closest to the 'ADVANCE_2030_Med2C' scenario and hence its `Emissions|VOC` pathway is returned.

In [ ]:
filler_input_filled.timeseries()

In [ ]:
sr15_data_closest_rms.filter(scenario="ADVANCE_2030_Med2C", variable="Emissions|VOC").timeseries()

### Rolling Window Quantile cruncher

This cruncher uses the 'rolling window quantile' methodology. It is described by its docstring.

In [ ]:
print(silicone.database_crunchers.DatabaseCruncherQuantileRollingWindows.__doc__)
# silicone.database_crunchers.DatabaseCruncherQuantileRollingWindows?

#### Infilling

Because it relies on quantile analysis, this database does best with large amounts of data. Hence we keep quite a lot of emissions timeseries before beginning.

In [ ]:
sr15_data_quantile = sr15_data.copy().filter(
    model=["WITCH-GLOBIOM 4.2"], variable=["Emissions|CO2", "Emissions|VOC"]
)

In [ ]:
cruncher = silicone.database_crunchers.DatabaseCruncherQuantileRollingWindows(sr15_data_quantile)

In [ ]:
filler = cruncher.derive_relationship("Emissions|VOC", ["Emissions|CO2"])
print(filler.__doc__)
# filler?

Now we can infill.

In [ ]:
filler_data = sr15_data.filter(
    model="MESSAGEix-GLOBIOM 1.0", 
    scenario="CD-LINKS_NPi2020_1600", 
    variable=["Emissions|CO2"]
)

The `filler_data` is missing `Emissions|VOC`.

In [ ]:
filler_data.filter(variable="Emissions|VOC").data

The rolling windows quantile can only downscale on the timesteps in the database. If this isn't the case, we will receive a `ValueError`.

In [ ]:
try:
    filler(filler_data)
except ValueError:
    traceback.print_exc(limit=0, chain=False)

We can fix this by filtering first.

In [ ]:
# NBVAL_IGNORE_OUTPUT
filler_data = filler_data.filter(year=sr15_data_quantile["year"].unique())

In [ ]:
filler_data.timeseries()

In [ ]:
filler_data_filled = filler(filler_data)
filler_data_filled.head()

In [ ]:
pkwargs = {"color": "scenario", "linewidth": 2}

fig = plt.figure(figsize=(16, 9))
ax = fig.add_subplot(121)
sr15_data_quantile.filter(variable="*CO2").line_plot(ax=ax, **pkwargs)
# sr15_data_closest_rms.filter(variable="*CO2", **example_model_scen).line_plot(ax=ax, color="black")
filler_data.filter(variable="*CO2").line_plot(ax=ax, color="black", linestyle="--", dashes=(10, 15))
ax.set_title("Database CO$_2$ and lead emissions timeseries")

ax = fig.add_subplot(122)
sr15_data_quantile.filter(variable="*VOC").line_plot(ax=ax, **pkwargs)
filler_data_filled.filter(variable="*VOC").line_plot(ax=ax, color="black", linestyle="--", dashes=(10, 15))
ax.set_title("Database VOC and filled follower emissions timeseries")

plt.tight_layout()

#### Recovering the data you put in

Because of the averaging mechanism, the cruncher will only precisely reproduce the input data when only one timeseries is put into it. With multiple data series, this is not guaranteed, although will happen in the limit case of many windows. 

This section does not illustrate new functionality, merely investigates the mathematical effects of changing the number of windows.

In [ ]:
# NBVAL_IGNORE_OUTPUT
# Multiple data are input - the 50th percentile and nwindows=10 is the default
filler_data = sr15_data_quantile.filter(scenario='ADVANCE_2030_Med2C')
self_cruncher = silicone.database_crunchers.DatabaseCruncherQuantileRollingWindows(sr15_data)
self_filler_default = self_cruncher.derive_relationship("Emissions|VOC", ["Emissions|VOC"], nwindows=10)
infilled_quantile = self_filler_default(filler_data)
infilled_quantile.head()

The default behaviour shows the correct trend, but the windows-based interpolation introduces some noise.

In [ ]:
# NBVAL_IGNORE_OUTPUT
pkwargs = {"color": "scenario", "linewidth": 2}
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot()
filler_data.filter(variable='Emissions|VOC', ).line_plot(ax=ax, **pkwargs)
infilled_quantile.filter(variable='Emissions|VOC').line_plot(ax=ax, color="black", linestyle="--", dashes=(8, 10))

By increasing the windows number, we can get a higher fidelity - at the cost of more computational power and less averaging over different datapoints. 

In [ ]:
# NBVAL_IGNORE_OUTPUT
# With a lower window number, the deviations become clearer
self_filler_low_w = self_cruncher.derive_relationship("Emissions|VOC", ["Emissions|VOC"], nwindows=2)
infilled_low_w = self_filler_low_w(filler_data)
infilled_low_w.head()

In [ ]:
# NBVAL_IGNORE_OUTPUT
# Higher window numbers mean that we weight data with the exact same x-value more highly, recovering the original relation
self_filler_high_w = self_cruncher.derive_relationship("Emissions|VOC", ["Emissions|VOC"], nwindows=30)
infilled_high_w = self_filler_high_w(filler_data)
infilled_high_w.head()

In [ ]:
# NBVAL_IGNORE_OUTPUT
# Here the lower window count (orange) results in large deviations whereas the high window case (red) provides a close match. 
pkwargs = {"color": "scenario", "linewidth": 2}
fig = plt.figure(figsize=(16, 9))
ax = fig.add_subplot(121)
# This line illustrates the background data informing the deviations (but confuses the plots). 
# Re-add it in place of the legend line if desired
#sr15_data.filter(variable='Emissions|VOC',).line_plot(ax=ax, linewidth=0.1)

filler_data.filter(variable='Emissions|VOC', ).line_plot(ax=ax, **pkwargs)
infilled_quantile.filter(variable='Emissions|VOC').line_plot(ax=ax, color="black", linestyle="--", dashes=(8, 10))
infilled_low_w.line_plot(ax=ax, color="blue", linestyle="--", dashes=(7, 8))
infilled_high_w.line_plot(ax=ax, color="red", linestyle="--", dashes=(7, 8))
plt.legend(['Original', 'n=10', 'n=2', 'n=30'])

### Time-dependent ratio cruncher
This cruncher uses the 'time-dependent ratio' methodology. It is described by its docstring.

In [ ]:
print(silicone.database_crunchers.DatabaseCruncherTimeDepRatio.__doc__)

#### Infilling
This technique is extracts only a single constant multiplier from all data fed in to the cruncher, so additional data does not radically change how it behaves. We will use the same amount of data as in the previous case for comparison. This cruncher works well for cases like splitting groups of gases into constituent parts, but poorly for gases which may have negative emissions. 

In [ ]:
sr15_data_tdr = sr15_data.copy().filter(
    model=["WITCH-GLOBIOM 4.2"], 
    variable=["Emissions|CO2", "Emissions|VOC", "Emissions|CH4" ,"Emissions|HCF*"]
)

In [ ]:
cruncher = silicone.database_crunchers.DatabaseCruncherTimeDepRatio(sr15_data_tdr)
filler = cruncher.derive_relationship("Emissions|VOC", ["Emissions|CO2"])
print(filler.__doc__)

In [ ]:
# NBVAL_IGNORE_OUTPUT
filler_data = sr15_data.filter(
    model="MESSAGEix-GLOBIOM 1.0", 
    scenario="CD-LINKS_NPi2020_1600", 
    variable=["Emissions|CO2"]
)
filler_data = filler_data.filter(year=sr15_data_quantile["year"].unique())
filler_data.data

In [ ]:
# NBVAL_IGNORE_OUTPUT
results= filler(filler_data)
results.head()

We observe that the results become anomalous when the lead emissions may be negative:

In [ ]:
pkwargs = {"color": "scenario", "linewidth": 2}

fig = plt.figure(figsize=(16, 9))
ax = fig.add_subplot(121)
sr15_data_tdr.filter(variable="*CO2").line_plot(ax=ax, **pkwargs)
filler_data.filter(variable="*CO2").line_plot(ax=ax, color="black", linestyle="--", dashes=(10, 15))
ax.set_title("Database CO$_2$ and lead emissions timeseries")

ax = fig.add_subplot(122)
sr15_data_tdr.filter(variable="*VOC").line_plot(ax=ax, **pkwargs)
results.filter(variable="*VOC").line_plot(ax=ax, color="black", linestyle="--", dashes=(10, 15))
ax.set_title("Database VOC and filled follower emissions timeseries")

plt.tight_layout()

#### HFC emission splitting
One of the features of the time-dependent ratio is that if the constituent elements consistently add up to a particular value, the infilled results will too, i.e. if for gases A, B, C we have A = B + C in all source material, then this will hold for the infilled values too. This makes it particularly appropriate for splitting gases into their constituents. 

In [ ]:
sr15_data_hfc = sr15_data.filter(variable="Emissions|HFC*")
sr15_data_hfc.variables()

In [ ]:
# We don't have all variables for all scenarios, but by finding a scenario where we do we can make progress
common_models = set(sr15_data_hfc.filter(variable="Emissions|HFC|HFC245fa").models()).intersection(
    set(sr15_data_hfc.filter(variable="Emissions|HFC").models())
)
common_scenarios = set(sr15_data_hfc.filter(variable="Emissions|HFC|HFC245fa", model=common_models).scenarios()).intersection(
    set(sr15_data_hfc.filter(variable="Emissions|HFC", model=common_models).scenarios())
)
infill_scenarios = "GEA_Eff_1p5C"
#set(sr15_data_hfc.filter(variable="Emissions|HFC").scenarios()) - set(sr15_data_hfc.filter(variable="Emissions|HFC|HFC245fa").scenarios())

infill_models ='MESSAGE V.3' 
# set(sr15_data_hfc.filter(variable="Emissions|HFC").models()) - set(sr15_data_hfc.filter(variable="Emissions|HFC|HFC245fa").models())

infill_scenarios

In [ ]:
common_years = np.arange(2010, 2105, 10)
hfc_training_data = sr15_data_hfc.filter(model=common_models, scenario=common_scenarios, year=common_years)
cruncher = silicone.database_crunchers.DatabaseCruncherTimeDepRatio(hfc_training_data)
filler = cruncher.derive_relationship("Emissions|HFC|HFC245fa", ["Emissions|HFC"])

In [ ]:
to_infill = sr15_data_hfc.filter(model=infill_models, scenario=infill_scenarios, year=common_years)
infilled = filler(to_infill)
infilled.head()

In [ ]:
# NBVAL_IGNORE_OUTPUT
pkwargs = {"color": "scenario", "linewidth": 2}

fig = plt.figure(figsize=(16, 9))
ax = fig.add_subplot(121)
hfc_training_data.filter(variable="*HFC").line_plot(ax=ax, **pkwargs)
to_infill.filter(variable="*HFC").line_plot(ax=ax, color="black", linestyle="--", dashes=(10, 15))
ax.set_title("Database HFC and lead emissions timeseries")

ax = fig.add_subplot(122)
hfc_training_data.filter(variable="*HFC245fa").line_plot(ax=ax, **pkwargs)
infilled.filter(variable="*HFC245fa").line_plot(ax=ax, color="black", linestyle="--", dashes=(10, 15))
ax.set_title("Database HFC245fa and filled follower emissions timeseries")

plt.tight_layout()

### Constant ratio cruncher
This infiller simply multiples a known value by a constant, which it must be told. It must also be told the units of the value it returns. It does not require an input dataframe. 

In [ ]:
print(silicone.database_crunchers.DatabaseCruncherTimeDepRatio.__doc__)

In [ ]:
cruncher = silicone.database_crunchers.DatabaseCruncherConstantRatio()
filler = cruncher.derive_relationship("Emissions|VOC", ["Emissions|CO2"], 0.5, "kt VOC/yr")
print(filler.__doc__)

In [ ]:
to_infill = sr15_data.filter(model=infill_models)
infilled = filler(to_infill)
infilled.head()

The values returned are simply the input, scaled by a factor. 

In [ ]:
# NBVAL_IGNORE_OUTPUT
pkwargs = {"color": "scenario", "linewidth": 2}

fig = plt.figure(figsize=(16, 9))
ax = fig.add_subplot(121)
infilled.line_plot(ax=ax, **pkwargs)
to_infill.filter(variable="*CO2").line_plot(ax=ax, color="black", linestyle="--", dashes=(1, 1))
ax.set_title("Database HFC and lead emissions timeseries")
